## SQLite
This application will read the mailbox data (mbox.txt) and count the number of email messages per organization (i.e. domain name of the email address) using a database with the following schema to maintain the counts. The data is https://www.py4e.com/code3/mbox.txt?PHPSESSID=093cf2ccc1ae79f0a7cbc2546f06bf36 which I will download as mbox.txt to use locally.

In [1]:
import sqlite3
conn = sqlite3.connect('emaildb.sqlite')
cur = conn.cursor()
# Drops a table titled 'Counts' if exists
cur.execute('DROP TABLE IF EXISTS Counts')
# Create a table titled 'Counts', where there are two columns, 'email' and 'count'
cur.execute('CREATE TABLE Counts (org TEXT, count INTEGER)')

In [2]:
# Opens the mbox.txt file, which we will read line by line
fh = open('mbox.txt')
for line in fh:
    # Sorts through each line and finds lines with emails in them, selecting the organization the emails use
    if not line.startswith('From: '): continue
    org = line.split()[1].split('@')[1]
    # Looks through database and returns table with email (from above) and count
    cur.execute('SELECT count FROM Counts WHERE org = ? ', (org,))
    # Fetches table of the email, which is a row. The row/table will either already contain an email and count, or be empty
    # because it is the first instance of the email.
    row = cur.fetchone()
    if row is None:
        cur.execute('''INSERT INTO Counts (org, count)
                VALUES (?, 1)''', (org,))
    else:
        cur.execute('UPDATE Counts SET count = count + 1 WHERE org = ?',
                    (org,))
# Commits changes we made to table
conn.commit()

It worked! Using DB Browser for SQLite, iupui.edu led with a count of 536. umich.edu came in second with a count of 491.